In [1]:
%matplotlib inline
import sys
print(sys.version)
import numpy as np
print(np.__version__)
import pandas as pd
print(pd.__version__)
import matplotlib.pyplot as plt
from pandas_datareader import data, wb
from datetime import timedelta

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

3.5.1 |Anaconda 2.4.1 (x86_64)| (default, Dec  7 2015, 11:24:55) 
[GCC 4.2.1 (Apple Inc. build 5577)]
1.11.0
0.18.0


In [2]:
hw = pd.read_csv('HomeWeatherData.csv')
del hw['Unnamed: 0']
hw.gmt = pd.to_datetime(hw.gmt)
hw.head()

,stationID,gmt,tempF,relhum,altibarom,team,isclosable,altitude,localbarom
0,KLGA,2015-10-11 04:51:00,57,62,30.07,mets,False,54,30.016
1,KLGA,2015-10-11 03:51:00,57,57,30.08,mets,False,54,30.026
2,KLGA,2015-10-11 02:51:00,57,55,30.08,mets,False,54,30.026
3,KLGA,2015-10-11 01:51:00,59,48,30.07,mets,False,54,30.016
4,KLGA,2015-10-11 00:51:00,60,46,30.09,mets,False,54,30.036


In [3]:
hw.dtypes

stationID             object
gmt           datetime64[ns]
tempF                  int64
relhum                 int64
altibarom            float64
team                  object
isclosable              bool
altitude               int64
localbarom           float64
dtype: object

In [4]:
pitches = pd.read_csv('all-stadiums_short.csv')
del pitches['Unnamed: 0']
pitches['tfs_zulu'] = pd.to_datetime(pitches['tfs_zulu'])
pitches['masterpitchID'] = pitches['stadium'] + ' ' + pitches['tfs_zulu'].apply(lambda x: str(x)) + ' '

In [5]:
pitches.columns
pitches.shape
pitches.dtypes

name                            object
player_id                        int64
pitch_type                      object
pitch_result                    object
atbat_result                    object
start_speed                    float64
z0                             float64
x0                             float64
pfx_x                          float64
pfx_z                          float64
px                             float64
pz                             float64
break_angle                    float64
break_length                   float64
spin_rate                      float64
spin_dir                       float64
zone                           float64
balls                            int64
strikes                          int64
outs                             int64
play                            object
game_date                       object
inning                           int64
inning_topbot                   object
tfs                              int64
tfs_zulu                d

In [12]:
team = 'yankees'    #will be ---> for team in hw['team'].unique():
timekey = pd.Series(pitches['masterpitchID'])
timekey.index = pitches['tfs_zulu']
timekey_team = timekey[timekey.map(lambda x: x.split(' ')[0] == team)].sort_values()
# timekey_team.sample(25)
# timekey.sample(25)
timekey_team.head(3)

tfs_zulu
2015-04-06 17:23:26    yankees 2015-04-06 17:23:26 
2015-04-06 17:28:19    yankees 2015-04-06 17:28:19 
2015-04-06 17:29:20    yankees 2015-04-06 17:29:20 
Name: masterpitchID, dtype: object

In [13]:
weathertimes_team = hw['gmt'][hw.team == team].sort_values()
weathertimes_team.head(3)

11007   2015-03-30 05:51:00
11006   2015-03-30 06:51:00
11005   2015-03-30 07:51:00
Name: gmt, dtype: datetime64[ns]

In [14]:
pitch_to_weathsamp = timekey_team.asof(weathertimes_team)
pitch_to_weathsamp[pitch_to_weathsamp.map(lambda x: type(x)==str)]  #removes NaN
# timekey.asof(selection)   map(lambda x: type(x)).

gmt
2015-04-06 17:51:00    yankees 2015-04-06 17:47:52 
2015-04-06 18:51:00    yankees 2015-04-06 18:47:36 
2015-04-06 19:51:00    yankees 2015-04-06 19:41:05 
2015-04-06 20:51:00    yankees 2015-04-06 19:55:42 
2015-04-06 21:51:00    yankees 2015-04-06 19:55:42 
2015-04-06 22:51:00    yankees 2015-04-06 19:55:42 
2015-04-06 23:51:00    yankees 2015-04-06 19:55:42 
2015-04-07 00:51:00    yankees 2015-04-06 19:55:42 
2015-04-07 01:51:00    yankees 2015-04-06 19:55:42 
2015-04-07 02:51:00    yankees 2015-04-06 19:55:42 
2015-04-07 03:51:00    yankees 2015-04-06 19:55:42 
2015-04-07 04:51:00    yankees 2015-04-06 19:55:42 
2015-04-07 05:51:00    yankees 2015-04-06 19:55:42 
2015-04-07 06:51:00    yankees 2015-04-06 19:55:42 
2015-04-07 07:51:00    yankees 2015-04-06 19:55:42 
2015-04-07 08:51:00    yankees 2015-04-06 19:55:42 
2015-04-07 09:51:00    yankees 2015-04-06 19:55:42 
2015-04-07 10:51:00    yankees 2015-04-06 19:55:42 
2015-04-07 11:51:00    yankees 2015-04-06 19:55:42 
2015-04-

In [233]:
#pitchmaster = pd.DataFrame(pitch_to_weathsamp)  #this will be the final final dbase
for team in hw['team'].unique():
    timekey = pd.Series(pitches['masterpitchID'], index=pitches['tfs_zulu'])
    timekey_team = timekey[timekey.map(lambda x: x.split(' ')[0] == team)].sort_values()
    weathertimes_team = hw['gmt'][hw.team == team].sort_values()
    pitch_to_weathsamp = timekey_team.asof(weathertimes_team)
    # last line is to merge the weather into the pitch dbase using the 
    #pitchmaster = pd.DataFrame(pitch_to_weathsamp)
    #pitchmaster = pd.merge(pitch_to_weathsamp, pitches, how='inner')